# Bring your own custom drift detector with Amazon SageMaker Model Monitor

This notebook shows how to:

* Host a machine learning model in Amazon SageMaker and capture inference requests, results, and metadata
* Build a Docker container to include your custom drift algorithms
* Monitor a live endpoint for detecting drifts
* Visualize the drift results

## Background

Amazon SageMaker provides every developer and data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker is a fully-managed service that encompasses the entire machine learning workflow. You can label and prepare your data, choose an algorithm, train a model, and then tune and optimize it for deployment. You can deploy your models to production with Amazon SageMaker to make predictions and lower costs than was previously possible.

In addition, Amazon SageMaker enables you to capture the input, output and metadata for invocations of the models that you deploy. It also enables you to bring your own metrics to analyze the data and monitor its quality. In this notebook, you learn how Amazon SageMaker enables these capabilities.

## Setup

To get started, make sure you have these prerequisites completed.

* Specify an AWS Region to host your model.
* An IAM role ARN exists that is used to give Amazon SageMaker access to your data in Amazon Simple Storage Service (Amazon S3). See the documentation for how to fine tune the permissions needed.
* Create an S3 bucket used to store the data used to train your model, any additional model data, and the data captured from model invocations. For demonstration purposes, you are using the same bucket for these. In reality, you might want to separate them with different security policies.

In [1]:
# Handful of configuration

import os
import boto3
import json
from sagemaker import get_execution_role, session

region= boto3.Session().region_name

sm_client = boto3.client('sagemaker')

role = "arn:aws:iam::342474125894:role/service-role/AmazonSageMaker-ExecutionRole-20190405T234154"
print("RoleArn: {}".format(role))

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket =  session.Session(boto3.Session()).default_bucket()
print("Demo Bucket: {}".format(bucket))
prefix = 'sagemaker/DEMO-ModelMonitor'

s3_capture_upload_path = f's3://{bucket}/{prefix}/datacapture'
s3_report_path = f's3://{bucket}/{prefix}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

RoleArn: arn:aws:iam::342474125894:role/service-role/AmazonSageMaker-ExecutionRole-20190405T234154
Demo Bucket: sagemaker-ap-southeast-1-342474125894
Capture path: s3://sagemaker-ap-southeast-1-342474125894/sagemaker/DEMO-ModelMonitor/datacapture
Report path: s3://sagemaker-ap-southeast-1-342474125894/sagemaker/DEMO-ModelMonitor/reports


### Upload train and test datasets, and model file to S3

The dataset is taken from [UCI Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/adult). The task is to predict whether income exceeds $50K/yr based on census data. We have split the dataset into train and test datasets. The model was trained using XGBoost and the model file is provided here. 

We need test datasets for calculating projected accuracy.

In [2]:
# Trained model
model_file = open("model/model.tar.gz", 'rb')

# Training data for model development (with header and label column)
train_file = open("data/train.csv", 'rb')
# Hold out test data
test_file = open("data/test.csv", 'rb')

s3_model_key = os.path.join(prefix, 'model.tar.gz')
s3_train_key = os.path.join(prefix, 'train.csv')
s3_test_key = os.path.join(prefix, 'test.csv')

boto3.Session().resource('s3').Bucket(bucket).Object(s3_model_key).upload_fileobj(model_file)
boto3.Session().resource('s3').Bucket(bucket).Object(s3_train_key).upload_fileobj(train_file)
boto3.Session().resource('s3').Bucket(bucket).Object(s3_test_key).upload_fileobj(test_file)

print("Success! You are all set to proceed.")

Success! You are all set to proceed.


## Bring your own custom model drift detection algorithm

In order to bring your own custom model drift detection algorithm, you need to do following things:
* Create custom detection algorithms. We have included algorithms under src folder
* Create a Docker container.
* Set enviornmental variables where the container can find the datacapture data from SageMaker Model Monitor. These variables have to match with the values we provide to monitor scheduler later.

In [3]:
!pygmentize Dockerfile

FROM python:3.8-slim-buster

RUN pip3 install pandas==1.1.4 numpy==1.19.4 scikit-learn==0.23.2 scipy==1.5.4 boto3==1.17.12

WORKDIR /home

COPY src/* /home/

ENTRYPOINT ["python3", "drift_detector.py"]


### Build the container and upload it to ECR

In [4]:
from docker_utils import build_and_push_docker_image

repository_short_name = 'custom-model-monitor'

image_name = build_and_push_docker_image(repository_short_name)

Building docker image custom-model-monitor from Dockerfile
$ docker build -t custom-model-monitor -f Dockerfile .
Sending build context to Docker daemon  40.24MB
Step 1/5 : FROM python:3.8-slim-buster
 ---> 6e66513aef39
Step 2/5 : RUN pip3 install pandas==1.1.4 numpy==1.19.4 scikit-learn==0.23.2 scipy==1.5.4 boto3==1.17.12
 ---> Using cache
 ---> 663d56544432
Step 3/5 : WORKDIR /home
 ---> Using cache
 ---> 12399f02e946
Step 4/5 : COPY src/* /home/
 ---> 2c9724be939c
Step 5/5 : ENTRYPOINT ["python3", "drift_detector.py"]
 ---> Running in 54a22c3e4b30
Removing intermediate container 54a22c3e4b30
 ---> b745ec2262ca
Successfully built b745ec2262ca
Successfully tagged custom-model-monitor:latest
Done building docker image custom-model-monitor
ECR repository already exists: custom-model-monitor
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remo

## Setup endoint and enable data capture

The data that is sent for inference to the endpoint needs to pre-processed before the XGBoost model can do prediction. Below code shows custom input handler for inference endpoint.

In [5]:
!pygmentize script/inference.py

#  Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License").
#  You may not use this file except in compliance with the License.
#  A copy of the License is located at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  or in the "license" file accompanying this file. This file is distributed
#  on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
#  express or implied. See the License for the specific language governing
#  permissions and limitations under the License.

import os
import pickle
import pathlib

from io import StringIO

import pandas as pd

import sagemaker_xgboost_container.encoder as xgb_encoders


script_path = pathlib.Path(__file__).parent.absolute()
with open(f'{script_path}/preprocess.pkl', 'rb') as f:
    preprocess = pickle.load(f) 


def input_fn(request_body, content_type):
    """
    The SageMaker XGBoost model server receives the request data body and

### Setting up model endpoint can take few minutes.

In [6]:
from sagemaker.xgboost.model import XGBoostModel
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig

model_url = f's3://{bucket}/{s3_model_key}'

xgb_inference_model = XGBoostModel(
    model_data=model_url,
    role=role,
    entry_point='inference.py',
    source_dir='script',
    framework_version='1.2-1',
)

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path)

predictor = xgb_inference_model.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
    serializer=CSVSerializer(),
    data_capture_config=data_capture_config)

-------------!

## Create monitoring schedule to detect drifts on hourly basis

Default Model monitor can be setup to monitor the inference on an hourly basis against the baseline metrics and violations. In this example, we are setting custom model monitor. For this purpose, we are using Boto3 calls directly to setup model monitor with the container we built above. Note that we need to setup input and output paths on the container. 

In [ ]:
s3_train_path = f's3://{bucket}/{s3_train_key}'
s3_test_path = f's3://{bucket}/{s3_test_key}'
s3_result_path = f's3://{bucket}/{prefix}/result/{predictor.endpoint_name}'

# Create endpoint monitor using custom container
# 0/15 * * * ? *
sm_client.create_monitoring_schedule(
    MonitoringScheduleName=predictor.endpoint_name,
    MonitoringScheduleConfig={
        'ScheduleConfig': {
            'ScheduleExpression': 'cron(0 * ? * * *)'
        },
        'MonitoringJobDefinition': {
            'MonitoringInputs': [
                {
                    'EndpointInput': {
                        'EndpointName': predictor.endpoint_name,
                        'LocalPath': '/opt/ml/processing/endpointdata'
                    }
                },
            ],
            'MonitoringOutputConfig': {
                'MonitoringOutputs': [
                    {
                        'S3Output': {
                            'S3Uri': s3_result_path,
                            'LocalPath': '/opt/ml/processing/resultdata',
                            'S3UploadMode': 'EndOfJob'
                        }
                    },
                ]
            },
            'MonitoringResources': {
                'ClusterConfig': {
                    'InstanceCount': 1,
                    'InstanceType': 'ml.c5.xlarge',
                    'VolumeSizeInGB': 10
                }
            },
            'MonitoringAppSpecification': {
                'ImageUri': image_name,
                'ContainerArguments': [
                    '--train_s3_uri',
                    s3_train_path,
                    '--test_s3_uri',
                    s3_test_path,
                    '--target_label',
                    'income'
                ]
            },
            'StoppingCondition': {
                'MaxRuntimeInSeconds': 600
            },
            'Environment': {
                'string': 'string'
            },
            'RoleArn': role
        }
    }
)

In [8]:
print("s3_train_path", s3_train_path)
print("s3_test_path", s3_test_path)
print("s3_result_path", s3_result_path)



s3_train_path s3://sagemaker-ap-southeast-1-342474125894/sagemaker/DEMO-ModelMonitor/train.csv
s3_test_path s3://sagemaker-ap-southeast-1-342474125894/sagemaker/DEMO-ModelMonitor/test.csv
s3_result_path s3://sagemaker-ap-southeast-1-342474125894/sagemaker/DEMO-ModelMonitor/result/sagemaker-xgboost-2021-08-12-13-54-00-173


## Start sending pre-configured traffic to endpoint

The cell below starts a thread to send some pre-configured traffic at a constant rate to the endpoint. The data points have been pre-conditioned to have drift, so that we can visualize it later. The traffic is sent for about 10 hours. If you like to stop the traffic, you need to stop the kernel to terminate this thread. 

In [11]:
from threading import Thread
from time import time, sleep

def invoke_endpoint(ep_name, file_name, runtime_client):
    pre_time = time()
    with open(file_name) as f:
        count = len(f.read().split('\n')) - 2 # Remove EOF and header
    
    # Calculate time needed to sleep between inference calls if we need to have a constant rate of calls for 10 hours
    ten_hours_in_sec = 10*60*60
    sleep_time = ten_hours_in_sec/count
    
    with open(file_name, 'r') as f:
        next(f) # Skip header
        
        for ind, row in enumerate(f):   
            start_time = time()
            payload = row.rstrip('\n')
            response = runtime_client(data=payload)
            
            # Print every 15 minutes (900 seconds). Rate of send * number of secs = number to sends
            if (ind+1) % int(count/ten_hours_in_sec*900) == 0:
                print(f'Finished sending {ind+1} records.')
            
            # Sleep to ensure constant rate. Time spent for inference is subtracted
            sleep(max(sleep_time - (time() - start_time), 0))
                
    print("Done!")
    
print(f"Sending test traffic to the endpoint {predictor.endpoint_name}. \nPlease wait...")

thread = Thread(target = invoke_endpoint, args=(predictor.endpoint, 'data/infer.csv', predictor.predict))
thread.start()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Sending test traffic to the endpoint sagemaker-xgboost-2021-08-12-13-54-00-173. 
Please wait...


## Clean up resources
* Monitor schedule - needs to deleted before deleting endpoint
* Delete endpoint
* Delete model

In [16]:
sm_client.list_monitoring_schedules(EndpointName=predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


{'MonitoringScheduleSummaries': [{'MonitoringScheduleName': 'sagemaker-xgboost-2021-08-12-13-54-00-173',
   'MonitoringScheduleArn': 'arn:aws:sagemaker:ap-southeast-1:342474125894:monitoring-schedule/sagemaker-xgboost-2021-08-12-13-54-00-173',
   'CreationTime': datetime.datetime(2021, 8, 12, 14, 0, 31, 617000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2021, 8, 12, 14, 0, 37, 795000, tzinfo=tzlocal()),
   'MonitoringScheduleStatus': 'Scheduled',
   'EndpointName': 'sagemaker-xgboost-2021-08-12-13-54-00-173',
   'MonitoringType': 'DataQuality'}],
 'ResponseMetadata': {'RequestId': 'a1dea2c1-e99d-471d-997a-f8f5b3c9669d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a1dea2c1-e99d-471d-997a-f8f5b3c9669d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '435',
   'date': 'Thu, 12 Aug 2021 15:05:58 GMT'},
  'RetryAttempts': 0}}

Finished sending 1000 records.
Finished sending 1250 records.


In [ ]:
# sm_client.delete_monitoring_schedule(MonitoringScheduleName=predictor.endpoint)

# predictor.delete_endpoint()
# predictor.delete_model()